<a href="https://colab.research.google.com/github/unizar-flav/AKiPa/blob/master/AkiPa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Kinetic Parameters

In [ ]:
#@title Environment
#@markdown You need to run this cell only once regardless of the number of datasets to be evaluated.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile #Necessary for compressing the files into zip
import csv


from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, output_file, save, show

from bokeh.palettes import linear_palette, Viridis256
from bokeh.models import Button, CustomJS, TabPanel, Tabs, Legend, Span, Label, Select
from bokeh.layouts import column
output_notebook()

from google.colab import files
from datetime import datetime
!git clone https://github.com/unizar-flav/KiPaD.git
from KiPaD.funcionesGenerales import  procesa, argLeastSquares, deriv_RK

def basic_plot(df, Title, x_axis, y_axis, width=1000, height=600):
  # Create a figure:
  p = figure (title=Title,
              x_axis_label= x_axis,
              y_axis_label= y_axis,
              width=width, height=height)
  # Define font sizes for the title, axes, and labels
  p.title.text_font_size = '20pt'
  p.xaxis.axis_label_text_font_size = '16pt'
  p.yaxis.axis_label_text_font_size = '16pt'
  p.xaxis.major_label_text_font_size = '12pt'
  p.yaxis.major_label_text_font_size = '12pt'

  p.scatter(df.iloc[:,0],df.iloc[:,1], size=8, color="blue" )

  return p


def linear_model (params, conc):
  # Extract the parameters from params
  k_on = params['k_on']
  k_off = params['k_off']

  return k_on*conc + k_off

def hyperbolic_model (params, conc):
  # Extract the parameters from params
  #k_on = params['k_on']
  #k_off = params['k_off']
  Kd = params['Kd']
  k_lim = params['k_lim']
  #k_rev = params['k_rev']

  #k_off = max(k_off, 0)
  #k_on = max(k_on, 0)
  #K_d=k_off/k_on

  result = ((conc*k_lim)/(conc + Kd)
            #+ k_rev
            )
  return result

# This can also be written as ((k_lim)/(1 + K_d/conc) + k_rev)


def basic_plot_with_fit(df,y_model, Title, x_axis, y_axis, width=1000, height=600):
  # Create a figure:
  p = figure (title=Title,
              x_axis_label= x_axis,
              y_axis_label= y_axis,
              width=width, height=height)
  # Define font sizes for the title, axes, and labels
  p.title.text_font_size = '20pt'
  p.xaxis.axis_label_text_font_size = '16pt'
  p.yaxis.axis_label_text_font_size = '16pt'
  p.xaxis.major_label_text_font_size = '12pt'
  p.yaxis.major_label_text_font_size = '12pt'

  p.scatter(df.iloc[:,0],df.iloc[:,1], size=8, color="blue" , legend_label ="Experimental data")
  p.line (df.iloc[:,0], y_model, line_width=2, color="red", legend_label= "Modelled data")
  return p

In [ ]:
#@title Upload files

# Upload the file and save in a dictionary
uploaded=files.upload()

# Obtain the uploaded file name from dictionary
file_name = list(uploaded.keys())

datos = pd.read_csv(file_name[0])
datos

In [ ]:
#@title Plot experimental data (k_obs)

k_obs_plot = basic_plot(datos, Title = f"k_obs vs [L] // {file_name[0]}",
                                         x_axis="[L] (μM)", y_axis ="k_obs (1/s)")
show(k_obs_plot)

In [ ]:
#@title Parameters

#@markdown **Fitting Mechanism:**
fitting = "Hyperbolic" #@param["Linear", "Hyperbolic"]
#@markdown ****

# Enter the constants for the reactions
#@markdown **Linear Model Constants:**
k_on = 1 #@param {type:"number"}
k_on_fixed = False  # @param {type: "boolean"}
#@markdown
k_off = 1 #@param {type:"number"}
k_off_fixed = False  # @param {type: "boolean"}
#@markdown ****
#@markdown **Hyperbolic Model Constants:**
Kd = 1 #@param {type:"number"}
Kd_fixed = False  # @param {type: "boolean"}
#@markdown
k_lim = 1 #@param {type:"number"}
k_lim_fixed = False  # @param {type: "boolean"}
#@markdown
#k_rev = 1 #@param {type:"number"}
#k_rev_fixed = False  # @param {type: "boolean"}
#@markdown





# Define rate constants and their fixed status in a dictionary
constants_data_l = {
    'k_on': (k_on, k_on_fixed),
    'k_off': (k_off, k_off_fixed),

}

constants_data_h = {
    'Kd': (Kd, Kd_fixed),
    'k_lim': (k_lim, k_lim_fixed)
    #,'k_rev': (k_rev, k_rev_fixed)
}


if fitting == "Linear":
  constants_data= constants_data_l
  model = linear_model
elif fitting == "Hyperbolic":
  constants_data= constants_data_h
  model = hyperbolic_model
else:
  print("String inputted for fitting is not valid")

# Initialize dictionaries for fixed and variable rate constants (these last ones are to be optimized later)
fixed_ks = {}
variable_ks = {}

# Loop over each rate constant and classify it into fixed or variable
for rate, (value, is_fixed) in constants_data.items():
    if is_fixed:
        fixed_ks[rate] = value
    else:
        variable_ks[rate] = value
print("Fixed Rate Constants:")
for key, value in fixed_ks.items():
    print(f"{key} = {value}")

print("\nVariable Rate Constants:")
for key, value in variable_ks.items():
    print(f"{key} = {value}")

initial_ks = {**fixed_ks, **variable_ks}

In [ ]:
#@title Procesa
initial_params= {**initial_ks }
initial_params_var= {**variable_ks}


#initial_params['n_species']=n_species # This dictionary contains the rate constants and the number of species
#initial_params['pathlength']= pathlength # We add pathlength to the dictionary to streamline the parameters onto
# a single dictionary

# Prepare the list of the parameters names to be opimized
nombrParVar = list(initial_params_var.keys())

# Find the minimum value in the entire DataFrame
#min_value = datos_approx_df.min().min()

#cotaInf = [[min_value for val in row] for row in S_matrix]

# Independent values (concentration) and dependent values (k_obs)
fKwargs = dict(conc = datos.iloc[:,0])





sol=procesa(argLeastSquares = argLeastSquares,
            dictParEstim = initial_params,
            nombrParVar = nombrParVar,
            f = model,
            fKwargs = fKwargs,
            Y = datos.iloc[:,1]
            #bounds=[cotaInf]
            )

In [ ]:
#@title Plot experimental data (k_obs) with model fitting
ad_parameters =sol ['parAjustados']

Model= model(ad_parameters, datos.iloc[:,0])
#print(Model)
k_obs_plot_with_fit = basic_plot_with_fit(datos, Model, Title = f"k_obs vs [L] with fit // {file_name[0]}",
                                         x_axis="[L] (μM)", y_axis ="k_obs (1/s)")
show(k_obs_plot_with_fit)

In [ ]:
#@title Export results
# @markdown Write the name for the zip file that contains the inputted and produced data.

# Now let's record all the data that it has been inputted and generated to export it.

# Initial data
datos.to_csv('Experimental_data.csv', index=False)

# Modelled data
datos_modelled=pd.DataFrame({
  'Concentration (μM)': datos.iloc[:,0],
  'k_obs (1/s)': Model})
datos_modelled.to_csv('Modelled_data.csv', index=False)




# Now let's save the plots

# Plot k_obs_plot
output_file("k_obs_plot.html")
save(k_obs_plot, title="k_obs_plot",)


# Plot k_obs_plot with model fitting
output_file("k_obs_plot_with_fit.html")
save(k_obs_plot_with_fit, title="k_obs_plot_with_fit",)


# Then we edit the data generated by the fitting to save and export them

# To save the results we create two dictionariesPara guardar los resultados vamos a crear dos diccionarios a partir de la solución de procesa
# y de los diccionarios de los parametros iniciales (fijos y variables)

# To save the results we create a dictinary that includes the initial parameters and the result from
# the procesa function (sol)

# List of keys to extract from the sol dictionary
keys= ['parAjustados', 'sdPar', 'R2', 'detalles']

partial= {key:sol.get(key, None) for key in keys}

Initial_params={'initialPar':initial_params}

results={**Initial_params, **partial}

print(results)

# Create CSV
with open('Fitting_result.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Add blank rows
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # Headers
    writer.writerow(['initialPar', '', '', 'adjustedPar', '', '', 'sdPar'])

    # Row 1:
    if 'k_on' in results['initialPar']:
        writer.writerow([
            'k_on', results['initialPar']['k_on'], '',
            'k_on', results['parAjustados']['k_on'], '',
            'k_on_std', results['sdPar'].get('k_on_std', '') if 'k_on' in initial_params_var else ''  # Display k_on_std only if it is a variable parameter
        ])

    if 'k_off' in results['initialPar']:
        writer.writerow([
            'k_off', results['initialPar']['k_off'], '',
            'k_off', results['parAjustados']['k_off'], '',
            'k_off_std', results['sdPar'].get('k_off_std', '') if 'k_off' in initial_params_var else ''  # Display k_off_std only if it is a variable parameter
        ])

    if 'Kd' in results['initialPar']:
        writer.writerow([
            'Kd', results['initialPar']['Kd'], '',
            'Kd', results['parAjustados']['Kd'], '',
            'Kd_std', results['sdPar'].get('Kd_std', '') if 'Kd' in initial_params_var else ''  # Display Kd_std only if it is a variable parameter
        ])

    if 'k_lim' in results['initialPar']:
        writer.writerow([
            'k_lim', results['initialPar']['k_lim'], '',
            'k_lim', results['parAjustados']['k_lim'], '',
            'k_lim_std', results['sdPar'].get('k_lim_std', '') if 'k_lim' in initial_params_var else ''  # Display k_lim_std only if it is a variable parameter
        ])


    # Add blank rows
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # R2 Section
    writer.writerow(['R2'])
    writer.writerow(['R2', results['R2']])
    writer.writerow([''] * 7)
    writer.writerow([''] * 7)

    # Detalles Section
    writer.writerow(['Details'])

    # x values (convert floats to strings to concatenate with 'x')
    writer.writerow(['x'] + [str(x) for x in results['detalles']['x']])

    # cost
    writer.writerow(['cost', results['detalles']['cost']])

    # Add blank rows
    writer.writerow([''] * 7)

    # fun values (split into multiple rows)
    fun_values = results['detalles']['fun']
    writer.writerow(['fun'] + [str(f) for f in fun_values[:6]])
    writer.writerow(['fun'] + [str(f) for f in fun_values[6:]])

    # Add blank rows
    writer.writerow([''] * 7)

    # jac values (split into multiple rows)
    for row in results['detalles']['jac']:
        writer.writerow(['jac'] + [str(v) for v in row])

    # Add blank rows
    writer.writerow([''] * 7)

    # grad values (convert floats to strings)
    writer.writerow(['grad'] + [str(g) for g in results['detalles']['grad']])

    # optimality
    writer.writerow(['optimality', results['detalles']['optimality']])

    # active_mask (convert integers to strings)
    writer.writerow(['active_mask'] + [str(a) for a in results['detalles']['active_mask']])

    # nfev, njev, status, message, success
    writer.writerow(['nfev', results['detalles']['nfev']])
    writer.writerow(['njev', results['detalles']['njev']])
    writer.writerow(['status', results['detalles']['status']])
    writer.writerow(['message', results['detalles']['message']])
    writer.writerow(['success', results['detalles']['success']])



# Then we proceed to save all the files and compressed them into a zip file

# Take the current date and hour ()
current_time = datetime.now().strftime("%d%m%Y%H%M%S")

# Define the prefix and create the complete name of the zip file
name ="kinetic_study_" #@param {type: "string"}
zip_filename = f"{name}{current_time}.zip"

# Create a zip file with the name written
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Add CSV files
    zipf.write('Experimental_data.csv')
    zipf.write('Modelled_data.csv')
    zipf.write ('Fitting_result.csv')
    # Add HTML files (Bokeh plots)
    zipf.write('k_obs_plot.html')
    zipf.write('k_obs_plot_with_fit.html')




# Download the zipped file
files.download(zip_filename)
